In [1]:
import pandas as pd
from aif360.metrics import ClassificationMetric
from aif360.datasets import StandardDataset
import numpy as np
import random

pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[inFairness]'
pip install 'aif360[Reductions]'


In [2]:
# Fake dataframe for exploration purposes, adapted from https://stackoverflow.com/questions/64506977/calculate-group-fairness-metrics-with-aif360/64543058#64543058 
df_testing = pd.DataFrame({
    'gender': [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
    'race':   [0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2],
    'experience' : [random.random() for _ in range(12)]
})

df_true_labels = pd.DataFrame({'income': [0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1]})
df_pred_labels = np.array([[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]], dtype=np.float64)

In [3]:
# true_labels = BinaryLabelDataset(df=pd.concat([df_testing, df_true_labels], axis=1))
# pred_labels = BinaryLabelDataset(df=pd.concat([df_testing, df_pred_labels], axis=1))

dataset_true = StandardDataset(pd.concat([df_testing, df_true_labels], axis=1), 
                          label_name='income', 
                          favorable_classes=[1],
                          protected_attribute_names=['gender', 'race'], 
                          privileged_classes=[[1], [1, 2]]
                          )

dataset_pred = dataset_true.copy()
dataset_pred.labels = df_pred_labels

In [16]:
print(dataset_true.protected_attribute_names)
print(dataset_true.privileged_protected_attributes)
print(dataset_pred.unprivileged_protected_attributes)

['gender', 'race']
[array([1.]), array([1., 2.])]
[array([0.]), array([0.])]
<class 'numpy.ndarray'>


In [21]:
with dataset_true.temporarily_ignore("race"):
    print(dataset_true.features)
    print(dataset_true.feature_names)

[[0.         0.         0.07249987]
 [0.         1.         0.82100953]
 [0.         2.         0.91335378]
 [0.         0.         0.07736028]
 [0.         1.         0.60975377]
 [0.         2.         0.34025462]
 [1.         0.         0.13092951]
 [1.         1.         0.82604172]
 [1.         2.         0.9152109 ]
 [1.         0.         0.30186319]
 [1.         1.         0.41480413]
 [1.         2.         0.77303823]]
['gender', 'race', 'experience']


Trying single attribute bias

In [5]:
attr = dataset_pred.protected_attribute_names[0]
attr

'gender'

In [6]:
idx = dataset_pred.protected_attribute_names.index(attr)
idx

0

In [7]:
privileged_groups =  [{attr:dataset_pred.privileged_protected_attributes[idx][0]}] 
privileged_groups

[{'gender': 1.0}]

In [8]:
unprivileged_groups = [{attr:dataset_pred.unprivileged_protected_attributes[idx][0]}]
unprivileged_groups

[{'gender': 0.0}]

In [9]:
class_metric = ClassificationMetric(dataset_true, dataset_pred, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)

In [10]:
class_metric.statistical_parity_difference()

-1.0

Trying multiple attribute bias 

In [11]:

privileged_groups = [{name : attribute for name, attribute in zip(dataset_true.protected_attribute_names, dataset_true.privileged_protected_attributes)}] 
privileged_groups

[{'gender': array([1.]), 'race': array([1., 2.])}]

In [12]:
unprivileged_groups = [{name : attribute for name, attribute in zip(dataset_true.protected_attribute_names, dataset_true.unprivileged_protected_attributes)}] 
unprivileged_groups

[{'gender': array([0.]), 'race': array([0.])}]

This causes an exception

In [13]:
# class_metric = ClassificationMetric(dataset_pred, dataset_pred, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
# class_metric.statistical_parity_difference()

In [14]:
dataset_true

               instance weights            features                 labels
                                protected attribute                       
                                             gender race experience       
instance names                                                            
0                           1.0                 0.0  0.0   0.072500    0.0
1                           1.0                 0.0  1.0   0.821010    0.0
2                           1.0                 0.0  2.0   0.913354    0.0
3                           1.0                 0.0  0.0   0.077360    0.0
4                           1.0                 0.0  1.0   0.609754    1.0
5                           1.0                 0.0  2.0   0.340255    1.0
6                           1.0                 1.0  0.0   0.130930    0.0
7                           1.0                 1.0  1.0   0.826042    1.0
8                           1.0                 1.0  2.0   0.915211    0.0
9                        